Grupo:
Arthur /
Fabiano /
Mateus

In [ ]:
!python -m pip install --upgrade pip
!pip install seqeval
!pip install -U sklearn_crfsuite
!pip install scikit-learn==1.3.2

!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=b3a33fd2ecc7aaf1f8bb9b7e7fd345bb226d1f6c387f3e3037083875cb6a599a
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 19.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
2023-12-12 17:45:11.742930: E tens

In [ ]:
import os
import numpy as np

from sklearn_crfsuite import CRF
from seqeval.metrics import classification_report

In [ ]:
!wget https://raw.githubusercontent.com/messias077/ner_pt/main/data/corpora/le_ner/train.conll
!wget https://raw.githubusercontent.com/messias077/ner_pt/main/data/corpora/le_ner/test.conll

--2023-12-12 17:45:28--  https://raw.githubusercontent.com/messias077/ner_pt/main/data/corpora/le_ner/train.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2142199 (2.0M) [text/plain]
Saving to: ‘train.conll’

train.conll         100%[===================>]   2.04M  --.-KB/s    in 0.08s   

2023-12-12 17:45:28 (27.0 MB/s) - ‘train.conll’ saved [2142199/2142199]

--2023-12-12 17:45:28--  https://raw.githubusercontent.com/messias077/ner_pt/main/data/corpora/le_ner/test.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43844

In [ ]:
!wget https://raw.githubusercontent.com/ulysses-camara/ulysses-ner-br/main/annotated-corpora/PL_corpus_conll/pl_corpus_categorias/train.txt
!wget https://raw.githubusercontent.com/ulysses-camara/ulysses-ner-br/main/annotated-corpora/PL_corpus_conll/pl_corpus_categorias/test.txt

--2023-12-12 17:45:28--  https://raw.githubusercontent.com/ulysses-camara/ulysses-ner-br/main/annotated-corpora/PL_corpus_conll/pl_corpus_categorias/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 779304 (761K) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>] 761.04K  --.-KB/s    in 0.06s   

2023-12-12 17:45:29 (12.0 MB/s) - ‘train.txt’ saved [779304/779304]

--2023-12-12 17:45:29--  https://raw.githubusercontent.com/ulysses-camara/ulysses-ner-br/main/annotated-corpora/PL_corpus_conll/pl_corpus_categorias/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.

## Função para leitura da base de dados no padrão BIO

In [ ]:
def read_corpus_file(corpus_file, delimiter='\t', ner_column=1):
    with open(corpus_file, encoding='utf-8') as file:
        lines = file.readlines()
    data = []
    words = []
    tags = []
    for line in lines:
        line = line.replace('\n', '')
        if line != '':
            if delimiter in line:
                fragments = line.split(delimiter)
                words.append(fragments[0])
                tags.append(fragments[ner_column])
        else:
            if len(words) > 1:
                data.append((words, tags))
            words = []
            tags = []
    return data

In [ ]:
# corpus_name = 'le_ner'
corpus_name = 'ulysses_ner'

report_dir = 'report/'

train_file = None
test_file = None

id_ner = 1
delimiter = ' '

if corpus_name == 'le_ner':
  train_file = '/content/train.conll'
  test_file = '/content/test.conll'
elif corpus_name == 'ulysses_ner':
  train_file = '/content/train.txt'
  test_file = '/content/test.txt'

print(f'\nCorpus: {corpus_name}')

report_dir = os.path.join(report_dir, corpus_name)

os.makedirs(report_dir, exist_ok=True)

train_data = read_corpus_file(train_file, delimiter=delimiter, ner_column=id_ner)
test_data = read_corpus_file(test_file, delimiter=delimiter, ner_column=id_ner)

print(f'\nTrain data: {len(train_data)}')
print(f'Test data: {len(test_data)}')

test_data_original = np.array(test_data, dtype=object)


Corpus: ulysses_ner

Train data: 2269
Test data: 523


In [ ]:
train_data[0]

(['Sala', 'das', 'Sessões', ',', 'em', 'de', 'de', '2019', '.'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATA', 'O'])

## Função que executa o pré-processamento do corpus usando a ferramenta Spacy

In [ ]:
import spacy

def data_preprocessing(data):
    nlp = spacy.load(name='pt_core_news_sm',
                     disable=['parser', 'ner', 'lemmatizer', 'textcat'])
    preprocessed_data = []
    for d in data:
        sentence = ' '.join(d[0])
        doc = nlp(sentence)
        pos_tags = [t.pos_ for t in doc]
        preprocessed_data.append((d[0], pos_tags, d[1]))
    return preprocessed_data

In [ ]:
train_data = data_preprocessing(train_data)

test_data = data_preprocessing(test_data)

In [ ]:
train_data[0]

(['Sala', 'das', 'Sessões', ',', 'em', 'de', 'de', '2019', '.'],
 ['PROPN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'ADP', 'ADP', 'NUM', 'PUNCT'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATA', 'O'])

## Funções usadas para extrair as features dos tokens e de seus vizinhos.

In [ ]:
def extract_sent_features(sentence):
    return [extract_features(sentence, i) for i in range(len(sentence))]


def extract_labels(sentence):
    return [label for _, _, label in sentence]


def extract_features(sentence, i):
    word = sentence[i][0]
    postag = sentence[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'word.islower()': word.islower(),
        'word[0].isupper()': word[0].isupper(),
        'word[0].islower()': word[0].islower(),
        'not word[0].isalnum()': not word[0].isalnum(),
        'not word.isalnum()': not word.isalnum(),
        'word.isalpha()': word.isalpha()
    }
    if i > 0:
        word1 = sentence[i - 1][0]
        postag1 = sentence[i - 1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:word.islower()': word1.islower()
        })
    else:
        features['BOS'] = True # BOS = Begin of Sentence
    if i > 1:
        word1 = sentence[i - 2][0]
        postag1 = sentence[i - 2][1]
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:postag[:2]': postag1[:2],
            '-2:word.islower()': word1.islower()
        })
    if i < len(sentence) - 1:
        word1 = sentence[i + 1][0]
        postag1 = sentence[i + 1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:word.islower()': word1.islower()
        })
    else:
        features['EOS'] = True # EOS = End of Sentence
    if i < len(sentence) - 2:
        word1 = sentence[i + 2][0]
        postag1 = sentence[i + 2][1]
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:postag[:2]': postag1[:2],
            '+2:word.islower()': word1.islower()
        })
    return features


def convert_data(data):
    sentences = []
    for d in data:
        sentences.append(list(zip(d[0], d[1], d[2])))
    x_data = [extract_sent_features(s) for s in sentences]
    y_data = [extract_labels(s) for s in sentences]
    return x_data, y_data

In [ ]:
X_train, y_train = convert_data(train_data)

X_test, y_test = convert_data(test_data)

In [ ]:
print(f'\nExample features: {X_train[0]}')

print(f'\nLabel: {y_train[0]}')


Example features: [{'bias': 1.0, 'word.lower()': 'sala', 'word[-3:]': 'ala', 'word[-2:]': 'la', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'PROPN', 'postag[:2]': 'PR', 'word.islower()': False, 'word[0].isupper()': True, 'word[0].islower()': False, 'not word[0].isalnum()': False, 'not word.isalnum()': False, 'word.isalpha()': True, 'BOS': True, '+1:word.lower()': 'das', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'ADP', '+1:postag[:2]': 'AD', '+1:word.islower()': True, '+2:word.lower()': 'sessões', '+2:word.istitle()': True, '+2:word.isupper()': False, '+2:postag': 'PROPN', '+2:postag[:2]': 'PR', '+2:word.islower()': False}, {'bias': 1.0, 'word.lower()': 'das', 'word[-3:]': 'das', 'word[-2:]': 'as', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'ADP', 'postag[:2]': 'AD', 'word.islower()': True, 'word[0].isupper()': False, 'word[0].islower()': True, 'not word[0].isalnum()': False, 'n

In [ ]:
crf = CRF(max_iterations=100, c1=0.1, c2=0.1, all_possible_transitions=True)

In [ ]:
try:
  crf.fit(X_train, y_train)
except AttributeError:
  pass

In [ ]:
y_pred = crf.predict(X_test)

In [ ]:
report = classification_report(y_test, y_pred)

print(report)

              precision    recall  f1-score   support

        DATA       0.96      0.94      0.95        98
      EVENTO       1.00      0.22      0.36         9
  FUNDAMENTO       0.85      0.85      0.85       124
       LOCAL       0.81      0.72      0.76       101
 ORGANIZACAO       0.76      0.72      0.74        94
      PESSOA       0.92      0.81      0.86       119
PRODUTODELEI       0.73      0.65      0.69        54

   micro avg       0.85      0.79      0.82       599
   macro avg       0.86      0.70      0.74       599
weighted avg       0.85      0.79      0.81       599



In [ ]:
report_file = os.path.join(report_dir, f'pln-trabalho-02-{corpus_name}.csv')
with open(report_file, 'w', encoding='utf-8') as f:
    f.write(report)

In [ ]:
def extract_ner(x, y, exclude=None, is_bio=False):
    if len(x) != len(y):
        print("\nERROR: 'x' and 'y' must be the same size.")
        return None
    if type(exclude) is not list:
        exclude = []  # Guarda os tokens que serão excluídos do resultado
    extracted_ners = []  # Guarda dicionários com as entidades nomeadas filtradas/agrupadas
    for i in range(len(y)):
        ind_last_token = len(y[i]) - 1
        j = 0
        same_entity = []  # Guarda os índices dos tokens vizinhos que pertencem à mesma entidade nomeada
        ners = {}  # Guarda o índice do token (como chave) e uma lista de tokens vizinhos que pertencem à mesma entidade
        has_began = False  # Indica se um label começou com 'B-', no caso do formato BIO
        # Percorre cada um dos tipos de tokens da sentença e, se houver, verifica se o proximo tipo de token é igual
        while j <= ind_last_token:
            if y[i][j] not in exclude:
                if j not in same_entity:
                    same_entity.append(j)
                    if is_bio and len(y[i][j]) >= 2 and y[i][j][:2].upper() == 'B-':  # Se é do formato BIO e é início
                        has_began = True
                if j + 1 <= ind_last_token:
                    if has_began and len(y[i][j + 1]) >= 2 and y[i][j + 1][:2].upper() == 'I-' \
                            and y[i][j][2:] == y[i][j + 1][2:]:
                        same_entity.append(j + 1)
                    elif not is_bio and y[i][j] == y[i][j + 1]:
                        same_entity.append(j + 1)
                    else:
                        ners[j] = same_entity
                        same_entity = []
                        has_began = False
                else:
                    ners[j] = same_entity
                    has_began = False
            j += 1
        # Processa as ners extraidas da sentença desta iteração
        if ners:
            sent = x[i]
            ners_aux = []
            for ind_token, filtered_tokens in ners.items():
                ner = ''
                for t in filtered_tokens:
                    ner += sent[t] + ' '
                # Trata o caso do formato BIO e retira os 'B-' e 'I-' dos nomes dos labels
                if is_bio and len(y[i][ind_token]) >= 2:
                    label = y[i][ind_token][2:]
                else:
                    label = y[i][ind_token]
                ners_aux.append((ner.strip(), label))
            extracted_ners.append(ners_aux)
        else:
            extracted_ners.append([])  # Para manter a ordem das sentenças, caso não seja extraida nenhuma ner
    return extracted_ners

In [ ]:
ner_ext_test = extract_ner(test_data_original[:, 0],test_data_original[:, 1], exclude=['O'], is_bio=True)

ner_ext_pred = extract_ner(test_data_original[:, 0], y_pred, exclude=['O'], is_bio=True)

In [ ]:
ner_ext_test[0]

[('art . 19 , da Lei n ° 6.001 , de 19 de dezembro de 1973', 'FUNDAMENTO')]

In [ ]:
ner_ext_pred[0]

[('art . 19 , da Lei n ° 6.001 , de 19 de dezembro de 1973', 'FUNDAMENTO')]